In [23]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import itertools
import matplotlib.pyplot as plt
import statistics
from nltk.corpus import wordnet as wn
from sklearn.preprocessing import StandardScaler
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric
import nltk
nltk.download('wordnet')
nltk.download('punkt')
import gensim.models
import re
from gensim.parsing.preprocessing import preprocess_string
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
import gensim.corpora as corpora
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.manifold import TSNE
!pip install -U spacy
import spacy

# Use Spacy as needed
nlp = spacy.load("en_core_web_sm")
import spacy
!pip install --upgrade gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
from google.colab import drive

# This will prompt you to click on a link and generate an authorization code
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
from google.colab import drive

# This will prompt you to click on a link and generate an authorization code
drive.mount('/content/drive')


full_dataframe = pd.read_excel("/content/drive/MyDrive/Thesis research /CPC Data/generated_data.xlsx", index_col=0)
full_dataframe = full_dataframe.dropna()
# full_dataframe=full_dataframe.drop(['dependent', 'sequence', 'claim_number', 'exemplary'], axis=1)
full_dataframe = full_dataframe[full_dataframe["labels"].str.contains('B60T')]
# full_dataframe = full_dataframe.head(20_000)
#import test
test_accepted = pd.read_excel("/content/drive/MyDrive/Thesis research /TRIZ Docs/Accepted_B60T.xlsx")
test_rejected = pd.read_excel("/content/drive/MyDrive/Thesis research /TRIZ Docs/Rejected_B60T.xlsx")
test = pd.concat([test_accepted, test_rejected], ignore_index=True)
num_rows = full_dataframe.shape[0]
error_df = pd.read_excel("/content/drive/MyDrive/Thesis research /Excels/Error Dataframe log.xlsx")
error_dict = dict(zip(error_df.patent_id, error_df.natural_log))
test["patent_id"] = test["patent_id"].astype(str)
test["natural_log"] = test["patent_id"].map(error_dict)

print("Columns before sorting:", test.columns)
test = test.sort_values(by='natural_log', ascending=False)
print("Columns after sorting:", test.columns)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columns before sorting: Index(['Unnamed: 0', 'patent_id', 'text', 'labels', 'anomaly?', 'text_vec',
       'label_vec', 'combined_vec', 'STD_combined_vec', 'natural_log'],
      dtype='object')
Columns after sorting: Index(['Unnamed: 0', 'patent_id', 'text', 'labels', 'anomaly?', 'text_vec',
       'label_vec', 'combined_vec', 'STD_combined_vec', 'natural_log'],
      dtype='object')


In [26]:
length_rejected = len(test_rejected)
# test = pd.concat([test_accepted, test_rejected], ignore_index=True)
# print(test.columns)
test = test.reset_index(drop=True)
test=test.drop(['anomaly?', 'text_vec', 'label_vec', 'combined_vec', 'STD_combined_vec', "natural_log"], axis=1)
len_test = len(test)
full_dataframe=pd.concat([full_dataframe,test], ignore_index=True)
copy_full_dataframe=full_dataframe.copy(deep=True)

In [27]:
for i in range (len(full_dataframe)):
    dropquote= eval(full_dataframe['labels'][i])
    # print(dropquote)
    tolist = list(dropquote)
    # print(tolist)
    full_dataframe['labels'][i] = tolist

In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))
# Remove Stop Words
data_words = list(sent_to_words(full_dataframe.text.values.tolist()))

data_words_nostops = remove_stopwords(data_words)


# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data=[]
for i in range (len(data_lemmatized)):
    data.append(' '.join(data_lemmatized[i]))

label_data_list = full_dataframe["labels"].tolist()
#create corpus of "sentances"

label_data = ([''.join(label) for label in label_data_list])
#
#remove \xa0
label_data = [el.replace(u'\xa0',' ') for el in label_data]

list_of_lists = [sentence.split() for sentence in data]


In [7]:
# from gensim.models import Word2Vec

# # Additional parameters for training
# vector_dim = 100  # Dimensionality of word vectors
# window_size = 5   # Context window size
# sg = 1            # Training algorithm: 1 for Skip-gram, 0 for CBOW
# negative_sampling = 5  # Number of negative samples
# min_word_count = 6     # Minimum word count threshold
# workers = 4            # Number of parallel workers for training

# # Train the Word2Vec model with additional parameters
# abstract_new_model = Word2Vec(
#     list_of_lists,
#     vector_size=vector_dim,
#     window=window_size,
#     sg=sg,
#     negative=negative_sampling,
#     min_count=min_word_count,
#     workers=workers
# )
# print(abstract_new_model)

# # Save the trained model
# abstract_new_model.save('model.bin')

# # Load the saved model
# abstract_new_model = Word2Vec.load('model.bin')
# print(abstract_new_model)


Word2Vec<vocab=3331, vector_size=100, alpha=0.025>
Word2Vec<vocab=3331, vector_size=100, alpha=0.025>


In [22]:
abstract_new_model = Word2Vec(list_of_lists, min_count=6)
print(abstract_new_model)
abstract_new_model.save('model.bin')
abstract_new_model = Word2Vec.load('model.bin')
print(abstract_new_model)

Word2Vec<vocab=3331, vector_size=100, alpha=0.025>
Word2Vec<vocab=3331, vector_size=100, alpha=0.025>


In [10]:
label_list_of_lists = [sentence.split("/") for sentence in label_data]
label_model = Word2Vec(label_list_of_lists, min_count=1)
print(label_model)
label_model.save('model.bin')
label_new_model = Word2Vec.load('model.bin')
print(label_new_model)
print(label_list_of_lists)
print("Vocabulary size:", len(abstract_new_model.wv))
print("Words in vocabulary:", abstract_new_model.wv.index_to_key[:10  ])  # Prin

Word2Vec<vocab=3865, vector_size=100, alpha=0.025>
Word2Vec<vocab=3865, vector_size=100, alpha=0.025>
[['F15B3', '00F15B7', '08B60T13', '147B60T13', '145'], ['G06F16', '29B60W2540', '26G06F7', '00B60W2530', '20B60T8', '172G08G1', '167A61B5', '18G06F16', '221B60W2552', '00B60W2540', '22B62D15', '025B60T2220', '02G08G1', '165H04L61', '2553B60C23', '04B60W10', '20B60W2554', '00G01C21', '3484B60W10', '184B60W50', '0098B60C23', '20B60W30', '18145G08G1', '166B60W30', '18B60T2210', '32B60C23', '0408H04L43', '08B60Q9', '00B60T2210', '13B60W2540', '221'], ['Y10T74', '20534B60W2420', '52B60W2554', '00B60W2420', '62B60W2050', '143B60W2520', '10B60W50', '16F02D11', '105B60W2540', '20B60T7', '22B60K26', '021B60W2554', '801G05G1', '30B60T17', '18B60W40', '02'], ['B60T17', '06B60T11', '26B60T8', '4081B60T11', '22'], ['B60T8', '885B60T2270', '414'], ['B60T13', '18'], ['B60W10', '18B60G2400', '302B60T2230', '02B60W2420', '22B60G2800', '213B60G17', '0162B60G2400', '64B60G2800', '922B60W40', '064B60T2240

In [28]:
word_vector = abstract_new_model.wv['vehicle']
print("Vector for 'word_to_check':", word_vector)

# Alternatively, you can check similarity between words
# similarity = abstract_new_model.wv.similarity('word1', 'word2')
# print("Similarity between 'word1' and 'word2':", similarity)

# You can also get the most similar words to a given word
most_similar_words = abstract_new_model.wv.most_similar('system', topn=5)
print("Most similar words to 'word_to_check':", most_similar_words)

Vector for 'word_to_check': [-0.93801737 -0.7384208   1.0926323   1.1093585  -0.08593923 -1.4238772
  0.48500872  1.482248   -1.4478004  -0.04563273 -0.04060022 -1.0956489
 -0.5596408  -0.17768712  1.1530163   0.45304805 -0.27347523 -0.6348137
  0.9608438   0.3525993   0.45186585 -0.22164863  0.19940573  0.5610302
  0.20336057  0.75087875  0.03369952 -0.4566279   0.63348156  1.6316853
  1.0500782  -0.08514843  1.3339272  -0.75779635  1.237848    1.2372824
 -0.16683333  1.0322788  -0.35113868  0.1594087   0.80223    -0.4037742
 -1.6054848   1.5929128   0.6296845   0.45191523  0.14772663  0.28320614
  0.4335956   0.02978318 -0.15949853 -0.5908769  -0.72582746  0.7006194
  0.5914896   0.7083487  -0.14917126  0.66437995  0.37041903  0.84157944
 -0.31200302  0.37881047 -0.5792501  -0.2321388  -0.51531786  0.47009107
  1.3558334  -0.05644825  0.7704838   0.8407563  -1.2206784  -0.85924894
 -0.9228371   1.4950879   0.04687227  0.14540589 -0.4491694  -0.03209382
  0.91116416 -0.12299023 -0.467

In [29]:
print("Vocabulary size:", len(label_new_model.wv))
print("Words in vocabulary:", label_new_model.wv.index_to_key[:10])

Vocabulary size: 3865
Words in vocabulary: ['B60T8', 'B60T13', 'B60T17', 'B60T7', 'B60T11', '00', '22', '02B60T8', '02', 'B60T2201']


In [30]:
word_vector = label_new_model.wv['B60T8']
print("Vector for 'word_to_check':", word_vector)

# Alternatively, you can check similarity between words
# similarity = label_new_model.wv.similarity('word1', 'word2')
# print("Similarity between 'word1' and 'word2':", similarity)
# You can also get the most similar words to a given word
most_similar_words = label_new_model.wv.most_similar('B60T8', topn=5)
print("Most similar words to 'word_to_check':", most_similar_words)

Vector for 'word_to_check': [-7.9346856e-04  5.7688897e-04  5.3615524e-03  9.1412729e-03
 -9.3885856e-03 -7.3124524e-03  6.5094591e-03  9.5747765e-03
 -5.2736448e-03 -3.7885236e-03  7.3309639e-03 -1.8965012e-03
 -4.5521134e-03  6.5671843e-03 -4.7281305e-03 -2.1611548e-03
  2.9128499e-03  5.2363076e-04 -8.4517561e-03 -9.9809095e-03
  7.6379096e-03  5.4670330e-03  7.4156434e-03  8.1504934e-04
  6.5417518e-03 -3.3338894e-03 -1.1775311e-03  6.0199355e-03
 -7.9404395e-03 -3.8696826e-03 -7.4872230e-03 -7.9125323e-04
  9.4658835e-03 -7.4348492e-03 -2.4957636e-03 -1.8440658e-03
  8.1899315e-03 -6.1204806e-03 -1.3241341e-05 -5.1117674e-03
 -9.5510185e-03  4.6401215e-03 -9.0599544e-03 -4.2479197e-03
  1.8945594e-04 -5.4302951e-04 -8.0147646e-03  9.7225299e-03
  5.4017748e-03  9.7846147e-03 -8.2257614e-03  4.3990910e-03
 -4.3232185e-03  6.1696797e-04  8.1624510e-03 -4.3003378e-03
  4.8820185e-03 -6.8061538e-03 -3.7328668e-03  9.5427558e-03
 -1.4105774e-03  5.3662405e-04 -4.4964198e-03 -7.5567947e

In [31]:
li=[]
for i in range(len(list_of_lists)):
  for j in range(len(list_of_lists[i])):
    li.append(list_of_lists[i][j])


In [32]:
print(len(li))

424369


In [33]:
list(set(li))

['screen',
 'unmanned',
 'phenomena',
 'improved',
 'adverse',
 'presently',
 'complete',
 'review',
 'tenon',
 'precrash',
 'gyroscope',
 'decide',
 'residence',
 'dish',
 'valuation',
 'multiplicity',
 'pincer',
 'melting',
 'plausibility',
 'accumulation',
 'rm',
 'driveably',
 'deformability',
 'sake',
 'underbrake',
 'separably',
 'membership',
 'bulky',
 'regeneratively',
 'perception',
 'download',
 'breathere',
 'cfr',
 'insensitive',
 'think',
 'reset',
 'absorption',
 'emulsion',
 'resilient',
 'slug',
 'condense',
 'lifting',
 'misjudge',
 'nowadays',
 'tofig',
 'unhindered',
 'uncommanded',
 'rage',
 'liftable',
 'fuzzy',
 'backflow',
 'resistor',
 'ethanol',
 'liquid',
 'overcurrent',
 'congestion',
 'effect',
 'hub',
 'undercutting',
 'moulded',
 'outlined',
 'closable',
 'urea',
 'initial',
 'exhaustion',
 'even',
 'reply',
 'attributable',
 'corrosive',
 'caliper',
 'dsc',
 'abnormality',
 'act',
 'truck',
 'inieee',
 'runaway',
 'trunk',
 'cab',
 'relaying',
 'pitch',


In [34]:
from gensim.models import Word2Vec
import json

# Load the pre-trained Word2Vec model
# abstract_new_model = Word2Vec.load('model.bin')

# List of words
word_list = li  # Replace this with your list of words

# Dictionary to store word vectors
word_vectors_dict = {}

# Iterate through the list of words
for word in word_list:
    print(word)
    try:
        # Get the vector representation of the word

        vector = abstract_new_model.wv[word]
        print("***")
        # Store the word and its vector in the dictionary
        word_vectors_dict[word] = vector.tolist()
    except KeyError:
        # If the word is not found in the model vocabulary, skip it
        print(f"Word '{word}' not found in the vocabulary.")

# Save the dictionary to a JSON file
with open('word_vectors.json', 'w') as json_file:
    json.dump(word_vectors_dict, json_file)

print("Word vectors saved to 'word_vectors.json'")


Streaming output truncated to the last 5000 lines.
***
assist
***
regulate
***
fluid
***
flow
***
master
***
cylinder
***
positive
***
displacement
***
pump
***
anti
***
lock
***
braking
***
system
***
background
***
invention
***
ability
***
independently
***
control
***
brake
***
force
***
vehicle
***
wheel
***
together
***
certain
***
special
***
sensor
***
enable
***
operation
***
vehicle
***
brake
***
system
***
various
***
special
***
mode
***
operation
***
special
***
mode
***
operation
***
lock
***
braking
***
mode
***
operation
***
commonly
***
refer
***
lock
***
brake
***
system
***
sensor
***
vehicle
***
brake
***
system
***
monitor
***
speed
***
vehicle
***
wheel
***
brake
***
brake
***
force
***
demand
***
brake
***
vehicle
***
wheel
***
cause
***
wheel
***
slip
***
brake
***
system
***
momentarily
Word 'momentarily' not found in the vocabulary.
reduce
***
brake
***
force
***
brake
***
wheel
***
allow
***
wheel
***
stop
***
slip
***
thus
***
provide
***
optimal
***
braking